<b> Problem Statement: </b> The objective of this Case is to Predication of bike rental count on daily based on the environmental and seasonal settings.


In [ ]:
#before starting this project with dataset. Initially I open given file in excel and just look the data available

#starting the project we need to load some libraries to deal with this data

import pandas as pd #for data processing & I/O operations
import numpy as np #for mathematical calculations
import seaborn as sns #for data visualization
import matplotlib.pyplot as plt #for plotting graphs

import sklearn #for machine learning algorithms

import os #for setting directory, I/O file operations

In [ ]:
#setting working directories
os.chdir("D:\Bike Renting")

In [ ]:
#checking the file directory
os.getcwd()

In [ ]:
#load required dataset
data = pd.read_csv("day.csv")

## Understanding the GIven Data

In [ ]:
#after loading dataset
#let's check the number of variables and observations in dataset
data.shape

after performning above function We see that therer are <b>731 observations(Rows) and 16 variables(Columns)</b> in given dataset 

In this <b>cnt</b> is our Target Variable and the others are predictor variables

In [ ]:
#now explore dataset more
#checking few dataset
data.head()

After checkingg few records of dataset, we get little bit confused about some of the variables values.

But don't worry in our problem statement, we get clarification about these variables and their values

<b>Important Note about Variables: </b>

* instant: record index

* dteday: date

* season: season (1:spring, 2:summer, 3:fall, 4:winter)

* yr: year (0: 2011, 1:2012)

* mnth: month ( 1 to 12)

* holiday: weather day is holiday or not (extracted from 

* weekday: day of the week

* workingday: if day is neither weekend nor holiday is 1, otherwise is 0.

* weathersit: (extracted fromFreemeteo) 
    * 1: Clear, Few clouds, Party cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 

* temp: Normalized temperature in Celsius. The values are divided via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale) 

- atemp: Normalized feeling temperature in Celsius. The values are divided via (t-t_min)/(t_max- t_min), t_min=-16, t_max=+50 (only in hourly scale) 

* hum: Normalized humidity. The values are divided to 100 (max)

* windspeed: Normalized wind speed. The values are divided to 67 (max)

* casual: count of casual users

* registered: count of registered users

* cnt: count of total rental bikes including both casual and registered


this shows that most of the variables are already converted as Categorical Variables using Normalization methods.

In [ ]:
#before moving further let's check the dtypes of each variables
data.dtypes

As we know, most of the variables are categorical, and season, yr, mnth, holiday, weekday, workingday, weathersit variables should be a categorical type, but they are int64. Now we need to convert them into catergorical variables 

In [ ]:
#converting to categorical variables

#so to convert muliple variables in one go, we need to create a loop function

#create a variable and store all variables
cat_var = ["season", "yr", "mnth", "holiday", "weekday", "workingday", "weathersit"]

for i in cat_var:
    data[i] = data[i].astype('category')

In [ ]:
#checking dtypes again
data.dtypes

In this dataset, some of the varaibles are not useful for further analysis for that reason we are dropping some of the variables here:

<b>Dropping Variables which are not required:</b>
* instant - index number, which is not useful in analysis
* dteday - all the required parameters are already extracted from this variable such as year, month, weekday. So this varaible is not useful

In [ ]:
#dropping instant and dteday variables
data = data.drop(['instant', 'dteday'], axis = 1)

In [ ]:
#checking the dataset after dropping two variables 
data.head()

## Missing Value Analysis

After converting dataset into proper format and dropping unuseful variables from dataset. Now its time to do Missing Value analysis

In [ ]:
#checking the missing values using isnull function

data.isnull().sum().sort_values(ascending= False)

There is no missing value present in given dataset

## Outlier Analysis

In [ ]:
#check summary of the dataset
data.describe()

Here, we use the <b>boxplot method</b> to visualize the outliers in our dataset

In [ ]:
#checking outliers in Target Variable "cnt" using boxplot method

#plotting boxplot for cnt variable
sns.boxplot(data =data, y="cnt", orient ='v')

From the above boxplots, it is evident that there is no outliers present in cnt.



In [ ]:
#let's check the outliers of predictor variables such as temp, atemp, hum, windspeed, casual, registered

fig, axes = plt.subplots(nrows=2, ncols= 3)

fig.set_size_inches(18,12)

#plotting boxplot of temp variable
sns.boxplot(data['temp'], orient ='v', ax=axes[0][0]).set_title("Boxplot of temp")

#plotting boxplot of atemp variable
sns.boxplot(data['atemp'], orient ='v', ax=axes[0][1]).set_title("Boxplot of atemp")

#plotting boxplot of hum variable
sns.boxplot(data['hum'], orient ='v', ax=axes[0][2]).set_title("Boxplot of hum")

#plotting boxplot of windspeed variable
sns.boxplot(data['windspeed'], orient ='v', ax=axes[1][0]).set_title("Boxplot of windspeed")

#plotting boxplot of casual variable
sns.boxplot(data['casual'], orient ='v', ax=axes[1][1]).set_title("Boxplot of casual")

#plotting boxplot of registered variable
sns.boxplot(data['registered'], orient ='v', ax=axes[1][2]).set_title("Boxplot of registered")

In [ ]:
#as we see that there are some outliers value present in 'hum', 'windspeed', 'casual' variables.

#before outlier removal lets findout the correlation analysis of these variables with target variables

print(data['hum'].corr(data['cnt']))

print(data['windspeed'].corr(data['cnt']))

print(data['casual'].corr(data['cnt']))

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(ncols= 3)

fig.set_size_inches(18,6)

#Correlation between 'hum' and 'cnt'  before  removal of  outliers
sns.regplot(x="hum", y="cnt", data=data, ax=ax1).set_title("Regression Plot of cnt vs hum")

#Correlation between 'windspeed' and 'cnt'  before  removal of  outliers
sns.regplot(x="windspeed", y="cnt", data=data, ax=ax2).set_title("Regression Plot of cnt vs windspeed")

#Correlation between 'casual' and 'cnt'  before  removal of  outliers
sns.regplot(x="casual", y="cnt", data=data, ax=ax3).set_title("Regression Plot of cnt vs casual")


<b>Outliers:</b> As we see from boxplot, correlation and regression plot, varaibles <b>hum, windspeed, casual</b> has ouliers and that have to be remove by outlier removal method  

In [ ]:
#make copy of dataset
df = data.copy()

In [ ]:
#Detect & Delete Outliers from the dataset

cnames = ['casual','hum','windspeed']

for i in cnames:
    q75, q25 = np.percentile(data.loc[:,i],[75,25])
    iqr = q75 - q25
    min = q25-(iqr*1.5)
    max = q75 +(iqr*1.5)
    print(min)
    print(max)
    data = data.drop(data[data.loc[:,i]< min].index)
    data = data.drop(data[data.loc[:,i]>max].index)

In [ ]:
#check shape of dataset
data.shape #58 obseravtions are dropped in outliers

## Correlation Analysis

<b>Correlation Analysis: </b> Here I am generating correlation matrix to understand how the each variable related with each other. In that I am plotting correlation matrix and generate plot using seabon library for better understanding 

In [ ]:
#generating correlation matrix
corr = data.corr()
corr

In [ ]:
#plotting correlation matrix and heatmap using seaborn libraty
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(corr,mask=np.zeros_like(corr, dtype=np.bool),cmap = sns.diverging_palette(220,10,as_cmap=True),square =True, annot=True, ax=ax)

<b>Correlation Analysis Result</b>
* temp and atemp are highly correlated
* temp and atemp have positive and strong correlation with cnt
* hum and windspeed have negative and weak correlation with cnt

In [ ]:
#dropping atemp variable from a dataset
data = data.drop(['atemp'], axis = 1)

In [ ]:
data.head(5)

##  Exploratory Data Analysis

In <b>Exploratory Data Analysis</b> we are going to find the how each predictor or variables related with Target Varaible:

* relation between Numerical Variable 'temp', 'hum', 'windspeed' and target variable 'cnt'


# Bivariate analysis

In [ ]:
#finding relationship between Numerical Variable 'temp', 'hum', 'windspeed' with target variable 'cnt'

fig, (ax1,ax2,ax3) = plt.subplots(ncols= 3)

fig.set_size_inches(15,9)

#relation between 'temp' and 'cnt'
sns.regplot(x="temp", y="cnt", data=data, ax=ax1).set_title("Regression Plot of cnt vs temp")

#relation between 'hum' and 'cnt'  
sns.regplot(x="hum", y="cnt", data=data, ax=ax2).set_title("Regression Plot of cnt vs hum")

#relation between 'windspeed' and 'cnt'  
sns.regplot(x="windspeed", y="cnt", data=data, ax=ax3).set_title("Regression Plot of cnt vs windspeed")

From the above plot, we see that 
* cnt has positive linear relationship with temp, 
* on the other side, cnt has a negative linear relationship with windspeed. 
* But hum(Humidity) has a little negative linear relationship with cnt.

In [ ]:
#now we find the relationship between categorical variables and Target Variable 'cnt'

# categorical variables are "Season", "holiday", "Weekday", "Workingday", "Weathersit", "month"
fig, axes = plt.subplots(nrows = 3, ncols=2)

fig.set_size_inches(12,18)

#plotting boxplot for cnt vs season variables
sns.boxplot(data =data, y="cnt", x="season", orient='v', ax=axes[0][0]).set_title("Boxplot of cnt vs season")

#plotting boxplot for cnt vs holiday variables
sns.boxplot(data =data, y="cnt", x="holiday", orient='v', ax=axes[0][1]).set_title("Boxplot of cnt vs holiday")

#plotting boxplot for cnt vs weekday variables
sns.boxplot(data =data, y="cnt", x="weekday", orient ='v', ax=axes[1][0]).set_title("Boxplot of cnt vs weekday")

#plotting boxplot for cnt vs workingday variables
sns.boxplot(data=data, y="cnt", x="workingday", orient='v', ax=axes[1][1]).set_title("Boxplot of cnt vs workingday")

#plotting boxplot for cnt vs month variables
sns.boxplot(data=data, y="cnt", x="mnth", orient='v', ax=axes[2][0]).set_title("Boxplot of cnt vs mnth")

#plotting boxplot for cnt vs Weathershit variables
sns.boxplot(data=data, y="cnt", x="weathersit", orient='v', ax=axes[2][1]).set_title("Boxplot of cnt vs weathersit")

<b>Results:</b>

* <b>Graph 1:cnt vs season</b>:     
    * cnt is very low in Spring Season and cnt is large in Fall Season 
 
 
* <b>Graph 2: cnt vs holiday</b> 
    * cnt is more on weekday i.e. no holiday
    
    
* <b> Gradph 3: cnt vs weekday</b>
    * as per the graph more number of bikes are used on Friday
    
    
* <b> Gradph 4: cnt vs workingday</b>
    * as per the graph more number of bikes are used on Workingday, this conclusion we already get from Graph 2
    
    
* <b> Gradph 5: cnt vs month</b>
    * as per the graph more number of bikes are used in July Month of year
    
    
* <b> Gradph 6: cnt vs Weathersit</b>
    * as per the graph more number of bikes are used when weather condition is Clear, Few clouds, Party cloudy, Partly cloudy
    * and less number of bikes are used when weather condition is Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * No bikes where used when weather condition is Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
    
    
    
<b>Summary:</b>
* cnt is maximum in good weather condition and minimum in bad weather condition
* more number of bikes are rented on weekdays.

## Feature Scalling 

as we know that, most of given variables are already Normalised

But let's clarify that using Visualization techniques

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 3)

fig.set_size_inches(15,12)

#Check whether  variable 'temp'is normal or not
sns.distplot(data['temp'], ax=axes[0][0]).set_title("Normalization of temp")

#Check whether  variable 'hum'is normal or not
sns.distplot(data['hum'], ax=axes[0][1]).set_title("Normalization of hum")

#Check whether  variable 'windspeed'is normal or not
sns.distplot(data['windspeed'], ax=axes[0][2]).set_title("Normalization of windspeed")

#Check whether  variable 'casual'is normal or not
sns.distplot(data['casual'],ax=axes[1][0]).set_title("Normalization of casual")

#Check whether  variable 'registered'is normal or not
sns.distplot(data['registered'], ax=axes[1][1]).set_title("Normalization of registered")

#as well as Check whether  Target variable 'cnt'is normal or not
sns.distplot(data['cnt'], ax=axes[1][2]).set_title("Normalization of Target Variable cnt")

As we can see, our data is in proper scalling form. Numerical Predictor Variables are Normalized

Also our Target Variable 'cnt' is also close to Normal Distribution

In [ ]:
data.shape

## Modeling & Prediction

Data Cleaning is done! Now lets bulid the model and predict the results

In machine Learning there is Two main types:
* <b>Supervised Machine Learning :</b> knowledge of output. Target Variable is fix 
* <b>Unsupervised Machine Learning:</b> No knowledge of Output. Self Guided Learnig Algorithms.

Selecting model is main Part of Modelling, We have various model algorithms some of the basic algorithms are:
* <b>Linear Regression :</b> Best suitable for Regression Model
* <b>Logistic Regression:</b> Suitable for Classification Model

* <b>Decision Tree:</b> Best suitable for Regression & Classification model
* <b>Random Forest:</b> Mostly used for Classification model analysis but can be use for Regression model

* <b>KNN algorithms: </b> Can be used for Regression and Classification model
* <b> Naive Bayes: </b> used for Classification Model

In our given dataset, the Target Variable 'cnt' is Numerical Continuous Variable. So we are dealing with <b>Regression Model</b> Analysis.

for that reason, we are considering following Algorithms:
* Linear Regression
* Decision Tree
* Random Forest
* KNN Algorithms

In [ ]:
# before moving further 
#let's droped the casual and registered variables because there sum is equal to target variable ie. 'cnt'

data = data.drop(['casual', 'registered'], axis =1)

In [ ]:
data.shape

In [ ]:
#now let's define the feature matrix and response vector
X = data.drop('cnt', axis=1)
y = data.iloc[:,-1].values

In [ ]:
#splitting X and y into training and testing dataset
#import sklearn train_test_split library 
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y =train_test_split(X,y,test_size=0.2)

## Linear Regression Model

In [ ]:
#train the model using traing dataset
#import LinearRegression libraries from sklearn
from sklearn import linear_model
import statsmodels.api as sm

In [ ]:
#train the model using training dataset
model_LR = sm.OLS(train_y, train_X.astype(float)).fit()

In [ ]:
model_LR.summary()

A few things, we learn from this output

* season, yr, weekday, workingday, weathersit, temp have small p-values, where as mnth, holiday, hum, windspeed have a larger p-values


* Here we reject the null-hypothesis for season, yr, weekday, workingday, weathersit, temp
    
    
    * There is assicoation between these variables and Target Variable cnt
        
        
* Fail to reject the null hypothesis for mnth, holiday, hum, windspeed

    * There is no association between these variables and Target Variable cnt
    

<b> R-squared (0.967)</b> means this model provides best fit for the given data 

but Selecting the model with the highest R-squared is not a reliable approach for choosing the best linear model.

<b>Solution:</b>

 * <b>Adjusted R-squared</b>
 
 Penalizes model complexity (to control for overfitting), but it generally under-penalizes complexity.
 
* <b>Better Solution:</b>

Do model Evaluation based on the Error Metrics for Regression: 

For classification problems, we have only used classification accuracy as our evaluation metric. But here we used Error Metrics to evaluate the model

<b>Mean Absolute Error (MAE):</b> is the mean of the absolute value of the errors: In [0,∞), the smaller the better


<b>Mean Squared Error (MSE):</b> is the mean of the squared errors: In [0,∞), the smaller the better

<b>Mean Absolute Percent Error (MAPE):</b> is the mean of the absolute percent value of the errors: In [0,1), the smaller the better


<b>Root Mean Squared Error (RMSE) :</b>is the square root of the mean of the squared errors: In [0,∞), the smaller the better

Let's calculate these by hand, to get an intuitive sense for the results:



In [ ]:
#make the predictions by model
predict_LR = model_LR.predict(test_X)

In [ ]:
def MAPE(true_y, pred_y):
    mape = np.mean(np.abs(true_y-pred_y)/true_y)
    return mape

In [ ]:
#importing important error metrics libraries
from sklearn.metrics import mean_absolute_error, mean_squared_error
# calculate MAE, MSE, MAPE, RMSE
print("MAE:",mean_absolute_error(test_y, predict_LR))
print("MSE:",mean_squared_error(test_y, predict_LR))
print("MAPE:",MAPE(test_y,predict_LR))
print("RMSE:",np.sqrt(mean_squared_error(test_y, predict_LR)))

* MAE gives less weight to outliers means it is not sensitive to outliers.
* MAPE is similar to MAE, but normalized the true obeservations. When true observation is zero then this metric will be problematic
* MSE is a combination measurement of bias and variance of predictions. It is more popular.
* RSME is square Root of MSE, Root square is taken to make the units of the error be the same as the units of the target. This measure gives more weight to large deviations such as outliers, since large differences squared become larger and small (smaller than 1) differences squared become smaller.

<b>Selection:</b> Outoff these 4 error metrics, MSE and RMSE are mainly used for Time-Series dataset. As we know, current working data is not a time dependend or time-series data. 

for that Reason the Model Evaluation is based on <b>MAPE Error Metrics</b>  


## Decision Tree Regression Model

In [ ]:
#decision tree regression
#import DecisionTreeRegressor Analysis

from sklearn.tree import DecisionTreeRegressor

In [ ]:
model_DT = DecisionTreeRegressor(max_depth = 2).fit(train_X,train_y)

In [ ]:
#apply the model on test data
predict_DT = model_DT.predict(test_X)

In [ ]:
# calculate MAE, MSE, MAPE, RMSE
print("MAE:",mean_absolute_error(test_y, predict_DT))
print("MSE:",mean_squared_error(test_y, predict_DT))
print("MAPE:",MAPE(test_y,predict_DT))
print("RMSE:",np.sqrt(mean_squared_error(test_y, predict_DT)))

## Random Forest Regression Model

In [ ]:
#Random forest analysis
#imnport RandomForestRegressor

from sklearn.ensemble import RandomForestRegressor

In [ ]:
##create Random Forest object
model_RF = RandomForestRegressor(n_estimators = 50)

##train the model using training dataset
model_RF.fit(train_X, train_y)

In [ ]:
#make the predictions by model
predict_RF = model_RF.predict(test_X)

In [ ]:
# calculate MAE, MSE, MAPE, RMSE
print("MAE:",mean_absolute_error(test_y, predict_RF))
print("MSE:",mean_squared_error(test_y, predict_RF))
print("MAPE:",MAPE(test_y,predict_RF))
print("RMSE:",np.sqrt(mean_squared_error(test_y, predict_RF)))

## KNN Regression Algorithms

In [ ]:
#KNN implementation
from sklearn.neighbors import KNeighborsRegressor


In [ ]:
model_KNN = KNeighborsRegressor(n_neighbors =5).fit(train_X, train_y)

In [ ]:
predict_KNN = model_KNN.predict(test_X)

In [ ]:
# calculate MAE, MSE, MAPE, RMSE
print("MAE:",mean_absolute_error(test_y, predict_KNN))
print("MSE:",mean_squared_error(test_y, predict_KNN))
print("MAPE:",MAPE(test_y,predict_KNN))
print("RMSE:",np.sqrt(mean_squared_error(test_y, predict_KNN)))

## Selecting Best Fit model for Future Analysis

We are cosidering the MAPE for model evaluatiomn becasue, it calculate average absolute percent error for each time period minus actual values divided by actual values. 

Reason we already discussed, let's explain again:

<b>Selection:</b> Outoff these 4 error metrics, MSE and RMSE are mainly used for Time-Series dataset. As I know, current working data is not a time dependend or time-series data.

<b>Random Forest </b> Model has smallest error metrics i.e.
* <b>MAPE = 0.1290541</b>

So, for further analysis We are selecting <b>Random Forest Model.</b>